# BLIP: Inference Demo
 - [Image Captioning](#Image-Captioning)
 - [VQA](#VQA)
 - [Feature Extraction](#Feature-Extraction)
 - [Image Text Matching](#Image-Text-Matching)

In [ ]:
# install requirements
import sys
if 'google.colab' in sys.modules:
    print('Running in Colab.')
    !pip3 install transformers==4.15.0 timm==0.4.12 fairscale==0.4.4
    !git clone https://github.com/salesforce/BLIP
    %cd BLIP

In [ ]:
from PIL import Image
import requests
import torch
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def load_demo_image(image_size,device):
    img_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg' 
    raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')   

    w,h = raw_image.size
    display(raw_image.resize((w//5,h//5)))
    
    transform = transforms.Compose([
        transforms.Resize((image_size,image_size),interpolation=InterpolationMode.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
        ]) 
    image = transform(raw_image).unsqueeze(0).to(device)   
    return image

# Image Captioning
Perform image captioning using finetuned BLIP model

In [ ]:
from models.blip import blip_decoder

image_size = 384
image = load_demo_image(image_size=image_size, device=device)

model_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_capfilt_large.pth'
    
model = blip_decoder(pretrained=model_url, image_size=image_size, vit='base')
model.eval()
model = model.to(device)

with torch.no_grad():
    # beam search
    caption = model.generate(image, sample=False, num_beams=3, max_length=20, min_length=5) 
    # nucleus sampling
    # caption = model.generate(image, sample=True, top_p=0.9, max_length=20, min_length=5) 
    print('caption: '+caption[0])

# VQA
Perform visual question answering using finetuned BLIP model

In [ ]:
from models.blip_vqa import blip_vqa

image_size = 480
image = load_demo_image(image_size=image_size, device=device)     

model_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_vqa_capfilt_large.pth'
    
model = blip_vqa(pretrained=model_url, image_size=image_size, vit='base')
model.eval()
model = model.to(device)

question = 'where is the woman sitting?'

with torch.no_grad():
    answer = model(image, question, train=False, inference='generate') 
    print('answer: '+answer[0])

# Feature Extraction

In [ ]:
from models.blip import blip_feature_extractor

image_size = 224
image = load_demo_image(image_size=image_size, device=device)     

model_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base.pth'
    
model = blip_feature_extractor(pretrained=model_url, image_size=image_size, vit='base')
model.eval()
model = model.to(device)

caption = 'a woman sitting on the beach with a dog'

multimodal_feature = model(image, caption, mode='multimodal')[0,0]
image_feature = model(image, caption, mode='image')[0,0]
text_feature = model(image, caption, mode='text')[0,0]

# Image-Text Matching

In [ ]:
from models.blip_itm import blip_itm

image_size = 384
image = load_demo_image(image_size=image_size,device=device)

model_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_retrieval_coco.pth'
    
model = blip_itm(pretrained=model_url, image_size=image_size, vit='base')
model.eval()
model = model.to(device='cpu')

caption = 'a woman sitting on the beach with a dog'

print('text: %s' %caption)

itm_output = model(image,caption,match_head='itm')
itm_score = torch.nn.functional.softmax(itm_output,dim=1)[:,1]
print('The image and text is matched with a probability of %.4f'%itm_score)

itc_score = model(image,caption,match_head='itc')
print('The image feature and text feature has a cosine similarity of %.4f'%itc_score)

In [1]:
#from models.blip_pretrain_ego4d import blip_pretrain_ego4d
import torch
from data.ego4d import Ego4dDataset

'''
def blip_pretrain_ego4d(**kwargs):
    model = BLIP_Ego4d(**kwargs)
    return model '''

'''num_frames=16
verb_classes=117
noun_classes=483
caption=["one","two","three","four","five","six","seven","eight","nine","ten","eleven","tweleve","thirteen","fourteen","fifteen","sixteen","one","two","three","four","five","six","seven","eight","nine","ten","eleven","tweleve","thirteen","fourteen","fifteen","sixteen"]
noun_labels=torch.tensor([2,3]) #"box"
verb_labels=torch.tensor([3,5]) #"put"'''



#model = blip_pretrain_ego4d(num_frames=num_frames, verb_classes=verb_classes, noun_classes=noun_classes, vision_width=512, med_config='configs/bert_config.json', embed_dim=256, queue_size=57600, momentum=0.995)

#out=model(caption, noun_labels, verb_labels,device,vid_feature=None)''''''


annots_dir_train = "/data/AmitRoyChowdhury/ego4d_data/v2/annotations/fho_lta_train.json"
annots_dir_test = "/data/AmitRoyChowdhury/ego4d_data/v2/annotations/fho_lta_test.json"
llava_captions_path = "/data/AmitRoyChowdhury/Anirudh/llava_object_responses/"
taxonomy_path = "/data/AmitRoyChowdhury/ego4d_data/v2/annotations/fho_lta_taxonomy.json"
short_annot_train_path="/data/AmitRoyChowdhury/ego4d_data/v2/annotations/fho_lta_train_short.json"
short_annot_test_path="/data/AmitRoyChowdhury/ego4d_data/v2/annotations/fho_lta_val_short.json"  #the path is correct, no worries

train_dataset = Ego4dDataset(mode='train',annots_path=annots_dir_train,taxonomy_path=taxonomy_path,llava_captions_path=llava_captions_path, short_annot_path=short_annot_train_path) 
#test_dataset = Ego4dDataset(mode='test',annots_path=annots_dir_test,taxonomy_path=taxonomy_path,llava_captions_path=llava_captions_path, short_annot_path=short_annot_test_path) 

print("dataset loaded")

for i,(vid,caption,ground_verb,ground_noun) in enumerate(train_dataset):
    print(vid)
    print(caption)
    print(ground_verb)
    print(groun_noun)
    exit()

total valid training files 63766
dataset loaded
llava file name /data/AmitRoyChowdhury/Anirudh/llava_object_responses/5f4558be-ecb4-4597-871f-cf1221301f91_start_frame_143_end_frame_383.pkl
/data/AmitRoyChowdhury/Anirudh/llava_object_responses/5f4558be-ecb4-4597-871f-cf1221301f91_start_frame_143_end_frame_383.pkl
['a person wearing blue gloves is kneading dough on a countertop, with a bowl of flour and a dough sheet', 'a person wearing blue gloves is kneading dough on a metal surface, with two bowls of dough and two pans', 'a person wearing blue gloves is kneading dough on a countertop, with a bowl of flour and a spoon nearby', 'a person wearing blue gloves is kneading dough on a table, with a bowl and a spoon nearby', 'a person wearing blue gloves is placing a piece of paper on a table, with a dough sheet and a', 'a person wearing blue gloves is placing a piece of dough on a table, with a knife and a spatula', 'a person wearing blue gloves is placing a piece of dough on a table, and th

NameError: name 'groun_noun' is not defined